In [2]:
%pwd

'c:\\Users\\shiva\\OneDrive\\Desktop\\Work\\NLP\\Projects\\Text_Summarizer_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\shiva\\OneDrive\\Desktop\\Work\\NLP\\Projects\\Text_Summarizer_Project'

Defining the entity:

In [26]:
from dataclasses import dataclass
from pathlib import Path

class DataTransformationConfig:
    def __init__(self,root_dir,data_path,tokenizer_name):

        self.root_dir =  root_dir
        self.data_path =  data_path
        self.tokenizer_name = tokenizer_name

Creating the configuration Manager:

In [27]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [28]:
class ConfigurationManager:
    
    #Defining a constructor with following paramteres:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):

        #Obtaining the config from the file_path:
        self.config = read_yaml(config_filepath)
        
        #Obtaining the paramters from the file_path:
        self.params = read_yaml(params_filepath)

        #Creating a new artifact: root folder:
        create_directories([self.config.artifacts_root])


    #A function to get data tranformation configuration:
    def get_data_tranformation_config(self) -> DataTransformationConfig:

        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_tranformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_tranformation_config

Creating Components:


In [29]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [30]:
class DataTransformation:

    def __init__(self, config: DataTransformationConfig):

        #Intialising the data ingestion config:
        self.config = config

        #Intializing the tokenizer:
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    #Converting the example text into features using tokenizer:
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True)

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels' : target_encodings['input_ids']
        }
    
    #A function to convert the features:
    def convert(self):

        #Obtaining the samsum dataset:
        dataset_samsum = load_from_disk(self.config.data_path)

        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched = True)

        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

Creating the pipeline:

In [31]:
try:
    #Initialising the configuration manager object:
    config = ConfigurationManager()

    #Obtaining the data tranformation:
    data_transformation_config = config.get_data_tranformation_config()

    # Passing the Obtained data tranformation into the Data_Tranformation Object:
    data_transformation = DataTransformation(config = data_transformation_config)

    #Calling the convert function:
    data_transformation.convert()
except Exception as e:
    raise e

[2025-03-28 19:42:40,943: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-28 19:42:40,947: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-28 19:42:40,949: INFO: common: created directory at: artifacts]
[2025-03-28 19:42:40,950: INFO: common: created directory at: artifacts/data_transformation]


c:\Users\shiva\OneDrive\Desktop\Work\NLP\Projects\Text_Summarizer_Project\textSummarization\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shiva\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? exampl